<a href="https://colab.research.google.com/github/Pavadareni/AI_Training/blob/main/Multi-lingual%20Sentiment%20Analysis/Militi_lingul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers langdetect torch fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296188 sha256=0c95bcb1b51021731ea9e9df669c66a34b406cf44dd6f55046f87b28657b1f4f
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [10]:
!git clone https://github.com/facebookresearch/fastText.git


Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (1026/1026), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 3998 (delta 890), reused 855 (delta 827), pack-reused 2972 (from 1)
Receiving objects: 100% (3998/3998), 8.30 MiB | 13.59 MiB/s, done.
Resolving deltas: 100% (2528/2528), done.


In [11]:
%cd fastText
!make


/content/fastText
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/vector.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/model.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/utils.cc
c++ -pthread -std=c++17 -march=native -O3 -funrol

In [12]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin -P ./fastText


--2024-11-19 07:08:23--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.164.78.128, 18.164.78.121, 18.164.78.81, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.164.78.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘./fastText/lid.176.bin’

lid.176.bin         100%[===================>] 125.18M   166MB/s    in 0.8s    

2024-11-19 07:08:24 (166 MB/s) - ‘./fastText/lid.176.bin’ saved [131266198/131266198]



In [13]:
!pip install fasttext


In [15]:
import fasttext

# Path to the downloaded model
model_path = "./fastText/lid.176.bin"

# Load the model
language_model = fasttext.load_model(model_path)

# Test language detection
text = "Bonjour tout le monde"
predictions = language_model.predict(text, k=1)  # Get top prediction
detected_language = predictions[0][0].replace("__label__", "")  # Clean label

print("Detected language:", detected_language)


Detected language: fr


In [16]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import torch
import fasttext

# Step 1: Load Pre-trained Sentiment Model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"  # Pretrained sentiment model
sentiment_model = XLMRobertaForSequenceClassification.from_pretrained(sentiment_model_path)
tokenizer = XLMRobertaTokenizer.from_pretrained(sentiment_model_path)

# Step 2: Load Pre-trained Language Detection Model
language_model_path = "./fastText/lid.176.bin"
language_model = fasttext.load_model(language_model_path)

# Step 3: Define Sentiment Labels
sentiment_labels = ["Negative", "Neutral", "Positive"]

# Step 4: Create Input Loop
print("Multilingual Sentiment Analysis with Language Detection")
print("Type 'exit' to quit the program.")

while True:
    user_input = input("\nEnter a sentence (any language): ")
    if user_input.lower() == "exit":
        print("Exiting... Goodbye!")
        break

    language_prediction = language_model.predict(user_input, k=1)  # Top-1 prediction
    language = language_prediction[0][0].replace("__label__", "").upper()

    print(f"Detected Language: {language}")

    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=128)

    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        sentiment_prediction = torch.argmax(outputs.logits, dim=-1).item()

    print(f"Sentiment: {sentiment_labels[sentiment_prediction]}")


Multilingual Sentiment Analysis with Language Detection
Type 'exit' to quit the program.

Enter a sentence (any language): Bonjour tout le monde
Detected Language: FR
Sentiment: Positive

Enter a sentence (any language): I love You
Detected Language: EN
Sentiment: Positive

Enter a sentence (any language): I hate you
Detected Language: EN
Sentiment: Negative

Enter a sentence (any language): உங்களை சந்திப்பதில் மகிழ்ச்சி அடைகிறேன்.
Detected Language: TA
Sentiment: Positive

Enter a sentence (any language): పుట్టినరోజు శుభాకాంక్షలు!
Detected Language: TE
Sentiment: Positive

Enter a sentence (any language): exit
Exiting... Goodbye!
